In [1]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
import webbrowser
import tempfile
import os

df = pd.read_csv(r"C:\CTSHack\MentalHealthProviders\PreprocessedFinal.csv")

geolocator = Nominatim(user_agent="mental_health_locator")


state_bboxes = {
    "alabama": [30.223334, 35.008028, -88.473227, -84.88908],
    "alaska": [51.214183, 71.365162, -179.148909, 179.77847],
    "arizona": [31.332177, 37.00426, -114.818269, -109.045223],
    "arkansas": [33.004106, 36.4996, -94.617919, -89.644395],
    "california": [32.534156, 42.009518, -124.409591, -114.131211],
    "colorado": [36.993076, 41.003444, -109.045223, -102.041524],
    "connecticut": [40.950943, 42.050587, -73.727775, -71.787239],
    "delaware": [38.451013, 39.839007, -75.789, -75.048939],
    "florida": [24.396308, 31.000968, -87.634938, -80.031362],
    "georgia": [30.357851, 35.000659, -85.605165, -80.839729],
    "hawaii": [18.910361, 22.2356, -160.2471, -154.806773],
    "idaho": [41.988057, 49.001146, -117.243027, -111.043564],
    "illinois": [36.970298, 42.508481, -91.513079, -87.494756],
    "indiana": [37.771742, 41.761368, -88.09776, -84.784579],
    "iowa": [40.37544, 43.501196, -96.639485, -90.140061],
    "kansas": [36.993083, 40.003162, -102.051744, -94.588413],
    "kentucky": [36.497129, 39.147458, -89.571509, -81.964971],
    "louisiana": [28.928609, 33.019457, -94.043147, -88.817017],
    "maine": [43.064941, 47.459686, -71.084843, -66.949895],
    "maryland": [37.911717, 39.723043, -79.487651, -75.048939],
    "massachusetts": [41.186328, 42.886589, -73.508142, -69.928393],
    "michigan": [41.696118, 48.2388, -90.418136, -82.413474],
    "minnesota": [43.499356, 49.384358, -97.239209, -89.491739],
    "mississippi": [30.147789, 34.996052, -91.655009, -88.097888],
    "missouri": [35.995683, 40.61364, -95.774704, -89.098843],
    "montana": [44.358221, 49.00139, -116.050003, -104.039138],
    "nebraska": [39.999998, 43.001708, -104.053514, -95.30829],
    "nevada": [35.001857, 42.002207, -120.005746, -114.039648],
    "new hampshire": [42.697042, 45.305476, -72.557247, -70.610621],
    "new jersey": [38.928519, 41.357423, -75.559614, -73.893979],
    "new mexico": [31.332301, 37.000232, -109.050173, -103.001964],
    "new york": [40.496103, 45.01585, -79.76259, -71.856214],
    "north carolina": [33.842316, 36.588117, -84.321869, -75.460621],
    "north dakota": [45.935054, 49.000239, -104.0489, -96.554507],
    "ohio": [38.403423, 41.977523, -84.820305, -80.518598],
    "oklahoma": [33.615833, 37.002311, -103.002565, -94.430662],
    "oregon": [41.991794, 46.292035, -124.566244, -116.463262],
    "pennsylvania": [39.7198, 42.26986, -80.519891, -74.689516],
    "rhode island": [41.146339, 42.018798, -71.862772, -71.12057],
    "south carolina": [32.033454, 35.215402, -83.35391, -78.499301],
    "south dakota": [42.479635, 45.94545, -104.057698, -96.436589],
    "tennessee": [34.982972, 36.678118, -90.310298, -81.6469],
    "texas": [25.837377, 36.500704, -106.645646, -93.508292],
    "utah": [36.997968, 42.001567, -114.052962, -109.041058],
    "vermont": [42.730315, 45.016659, -73.43774, -71.464555],
    "virginia": [36.540739, 39.466012, -83.675395, -75.242266],
    "washington": [45.543541, 49.002494, -124.848974, -116.916031],
    "west virginia": [37.201483, 40.638801, -82.644739, -77.719519],
    "wisconsin": [42.491983, 47.302537, -92.889433, -86.249548],
    "wyoming": [40.994746, 45.005904, -111.056888, -104.05216]
}

def show_providers_on_map(location_name, zoom_start=7):
    """
    Show providers for a given state (via bounding box) or city (via geopy bounding box).
    Directly opens map in browser without saving manually.
    """
    loc_lower = location_name.lower().strip()

    if loc_lower in state_bboxes:
        lat_min, lat_max, lon_min, lon_max = state_bboxes[loc_lower]
        filtered = df[
            (df["State"].str.lower() == loc_lower) &
            (df["Latitude"] >= lat_min) & (df["Latitude"] <= lat_max) &
            (df["Longitude"] >= lon_min) & (df["Longitude"] <= lon_max)
        ]
    else:

        try:
            location = geolocator.geocode(location_name, country_codes="US", exactly_one=True)
            if not location or "boundingbox" not in location.raw:
                print(f"❌ Could not find bounding box for {location_name}")
                return None

            bbox = location.raw["boundingbox"] 
            min_lat, max_lat = float(bbox[0]), float(bbox[1])
            min_lon, max_lon = float(bbox[2]), float(bbox[3])

            filtered = df[
                (df["Latitude"] >= min_lat) & (df["Latitude"] <= max_lat) &
                (df["Longitude"] >= min_lon) & (df["Longitude"] <= max_lon)
            ]
        except Exception as e:
            print(f"⚠️ Error geocoding {location_name}: {e}")
            return None

    if filtered.empty:
        print(f"❌ No providers found for {location_name}")
        return None

  
    center_lat, center_lon = filtered.iloc[0]["Latitude"], filtered.iloc[0]["Longitude"]

    m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start)

    for _, row in filtered.iterrows():
        popup_text = f"""
        <b>{row['Provider First Name']}</b><br>
        Gender: {row['gndr']}<br>
        Specialty: {row['pri_spec']}<br>
        Facility: {row['Facility Name']}<br>
        Phone: {row['Telephone Number']}<br>
        Address: {row['adr_ln_1']}, {row['City/Town']}, {row['State']} {row['ZIP Code']}
        """
        folium.Marker(
            location=[row["Latitude"], row["Longitude"]],
            popup=popup_text,
            icon=folium.Icon(color="green", icon="plus-sign")
        ).add_to(m)


    tmpfile = tempfile.NamedTemporaryFile(delete=False, suffix=".html")
    m.save(tmpfile.name)
    webbrowser.open(f"file://{os.path.abspath(tmpfile.name)}")

    print(f"✅ Showing map for {location_name} with {len(filtered)} providers")
    return filtered


user_input = input("Enter a state or city: ")
providers_found = show_providers_on_map(user_input, zoom_start=7)

if providers_found is not None:
    print(providers_found[["Provider First Name", "City/Town", "State", "pri_spec", "Telephone Number"]].head())


✅ Showing map for California with 252 providers
   Provider First Name     City/Town       State               pri_spec  \
12             MATTHEW     PALO ALTO  California             PSYCHIATRY   
13             MATTHEW      STANFORD  California             PSYCHIATRY   
16               SHAMA  CAMERON PARK  California  CLINICAL PSYCHOLOGIST   
17              JOSHUA   LOS ANGELES  California             PSYCHIATRY   
19              JOSHUA   LOS ANGELES  California             PSYCHIATRY   

    Telephone Number  
12      6.507234e+09  
13      6.507234e+09  
16      5.306774e+09  
17      3.108260e+09  
19      3.108259e+09  
